## Tutorial smolagent
https://huggingface.co/docs/smolagents/guided_tour?Pick+a+LLM=HF+Inference+API

In [4]:
!pip install smolagents python-dotenv 
#transformers accelerate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.5 MB 8.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.5 MB 5.1 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/11.5 MB 5.4 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/11.5 MB 4.4 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.5 MB 4.6 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.5 MB 4.6 MB/s eta 0:00:02
   ---------------------- ----------------- 6.6/11.5 MB 4.6 MB/s eta 0:00:02
   -------------------------- ------------- 7.6/11.5 MB 4.7 MB/s eta 0:00:01
   ------------------------------ --------- 8.7/11.5 MB 4.6 MB/s eta 0:00:01
   --------------------------------- ------ 9.7/11.5 MB 4.6 MB/s eta 0:00:01
   ------------------------------------ --- 10.5/11.5 MB 4.8 MB/s eta 0:00:01
   ------

In [5]:
!python.exe -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
from dotenv import load_dotenv
import os

# Cargar las variables de entorno del archivo .env
load_dotenv()

# Acceder a la clave privada
hf_key = os.getenv("HF_TOKEN")
#!huggingface-cli login --token $hf_key

In [7]:
model_id = "meta-llama/Llama-3.3-70B-Instruct" 
# model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

 ### 1. Usando HF Inference API

In [8]:
from smolagents import CodeAgent, HfApiModel

model = HfApiModel(model_id=model_id, token=hf_key) # You can choose to not pass any model_id to HfApiModel to use a default free model
# you can also specify a particular provider e.g. provider="together" or provider="sambanova"
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

e:\jaz\AI3\Academy\RetoSmolagent\Smol_Agent_Ferreteria\jaz\ejemplo1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      fib_sequence = [0, 1]                                                                                        
      while len(fib_sequence) < n + 1:                                                                             
          fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])                                                 
      return fib_sequence[n]                                                                                       
                                                                                                                   
  result = fibonacci(117)  # Note: sequence is 0-indexed                                                           
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
1264937032042997393488322

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(1264937032042997393488322)                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 1264937032042997393488322

[Step 1: Duration 2.96 seconds| Input tokens: 4,544 | Output tokens: 194]

1264937032042997393488322

El resultado a veces sale bien y otras no.

En el siguiente ejemplo, se define la función de fibonacci como una tool y entonces ya da el resultado correcto:

In [9]:
from smolagents import tool

@tool
def fibonacci(n: int) -> int:
    """
    Calcula el enésimo número de Fibonacci.

    Args:
        n: Posición del número de fibonacci a calcular.
    """

    a, b = 0, 1
    for _ in range(n - 1):
        a, b = b, a + b
    return a

model = HfApiModel(model_id=model_id, token=hf_key)

# Configurar el agente con la tool de Fibonacci
agent = CodeAgent(tools=[fibonacci], model=model, add_base_tools=True)

# Ejecutar la consulta al agente
response = agent.run("What is the 118th Fibonacci number?")

print("Response:", response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 118th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  fib_118 = fibonacci(n=118)                                                                                       
  print(fib_118)                                                                                                   
  final_answer(fib_118)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
1264937032042997393488322

Out - Final answer: 1264937032042997393488322

[Step 0: Duration 3.60 seconds| Input tokens: 2,171 | Output tokens: 59]

Response: 1264937032042997393488322


Otra forma de definir la tool pero usando una class

In [10]:
from smolagents import CodeAgent, HfApiModel
from smolagents.tools import Tool
import os

# Definir la clase Fibonacci heredando de Tool
class FibonacciTool(Tool):
    name = "fibonacci"  # Agregamos el atributo name
    description = "Calculates the nth Fibonacci number."
    inputs = {
        "n": {
            "type": "integer",
            "description": "The position in the Fibonacci sequence to calculate"
        }
    }
    output_type = "integer"  # Añadimos el tipo de salida
     
    def forward(self, n: int) -> int:
        """Calcula el enésimo número de Fibonacci."""
        a, b = 0, 1
        for _ in range(n - 1):
            a, b = b, a + b
        return a

# Crear una instancia de la herramienta
fibonacci_tool = FibonacciTool()


# Usar HfApiModel con una clave de Hugging Face
# hf_key = os.getenv("HF_TOKEN")  # Asegúrate de definir esta variable en tu entorno
# model_id = "meta-llama/Llama-3.3-70B-Instruct"  # Puedes cambiarlo por otro modelo disponible

model = HfApiModel(model_id=model_id, token=hf_key)

# Configurar el agente con la tool de Fibonacci
agent = CodeAgent(tools=[fibonacci_tool], model=model, add_base_tools=True)

# Ejecutar la consulta al agente
response = agent.run("What is the 118th Fibonacci number?")

print("Response:", response)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 118th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = fibonacci(n=118)                                                                                        
  print(result)                                                                                                    
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
1264937032042997393488322

Out - Final answer: 1264937032042997393488322

[Step 0: Duration 3.32 seconds| Input tokens: 2,166 | Output tokens: 57]

Response: 1264937032042997393488322


### Inspecting an agent run

Here are a few useful attributes to inspect what happened after a run:

agent.logs
Stores the fine-grained logs of the agent. At every step of the agent’s run, everything gets stored in a dictionary that then is appended to agent.logs.

Running agent.write_memory_to_messages() writes the agent’s memory as list of chat messages for the Model to view. This method goes over each step of the log and only stores what it’s interested in as a message: for instance, it will save the system prompt and task in separate messages, then for each step it will store the LLM output as a message, and the tool call output as another message. Use this if you want a higher-level view of what has happened - but not every log will be transcripted by this method.

In [11]:
agent.logs

The 'logs' attribute is deprecated and will soon be removed. Please use 'self.memory.steps' instead.


[SystemPromptStep(system_prompt='You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using 

### Utilidades

### 1. Buscar en internet conDuckDuckGo

In [12]:
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
print(search_tool("Who's the current president of Russia?"))

## Search Results

[List of presidents of Russia - Wikipedia](https://en.wikipedia.org/wiki/List_of_presidents_of_Russia)
The office of the president of Russia is the highest authority in the Russian Federation.The holder is the federation's head of state and has formal presidency over the State Council as well as being the commander in chief of the Russian Armed Forces.The office was introduced in 1918 after the February Revolution with the current office emerging after a referendum of 1991. [1]

[President of Russia - Wikipedia](https://en.wikipedia.org/wiki/President_of_Russia)
The president of Russia, officially the president of the Russian Federation (Russian: Президент Российской Федерации, romanized: Prezident Rossiyskoy Federatsii), is the executive head of state of Russia.The president is the chair of the Federal State Council and the supreme commander-in-chief of the Russian Armed Forces.It is the highest office in Russia.

[Biographies of presidents of Russia - President of 